In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U keras-tuner

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras import utils
import keras_tuner as kt

utils.set_random_seed(42)

In [ ]:
# Define Parameters
LOOKBACK = 24
HORIZON = 24
N_SPLITS = 4
EPOCHS = 10

# Prepare data

In [ ]:
imp_folder = os.getenv("DATA_PATH", "./default_data_path/")
exp_folder = os.getenv("MODEL_PATH", "./default_model_path/")

df = pd.read_csv(imp_folder + 'cell_multivar.csv')

print(df.shape)
df.columns

(933661, 24)


Index(['timestamp', 'cell', 'bts', 'antenna', 'carrier', 'minRSSI',
       'pageSessions', 'ULvol', 'sessionDur', 'blocks', 'AnomalyDay',
       'anomaly', 'noise', 'Height', 'Azimuth', 'SectorsPerBts', 'NearbyBts',
       'Dist2Coast', 'ClusterId', 'CellsPerBts', 'OverallPageSessions',
       'OverallULvol', 'OverallSessionDur', 'OverallBlocks'],
      dtype='object')

In [ ]:
temporal_X = ['pageSessions', 'ULvol', 'sessionDur']
static_X = ['Height', 'Azimuth', 'Dist2Coast', 'ClusterId',
            'CellsPerBts', 'OverallPageSessions', 'OverallULvol',
            'OverallSessionDur', 'OverallBlocks']

# Funcs

In [ ]:
# Time series split function (Expanding Window)
def time_series_split(df, n_splits=N_SPLITS, test_size=0.2):
    df = df.sort_values('timestamp')
    test_split_index = int(len(df) * (1 - test_size))
    train_val_df = df.iloc[:test_split_index]
    test_df = df.iloc[test_split_index:]

    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = [(train_val_df.iloc[train_index], train_val_df.iloc[val_index]) for train_index, val_index in tscv.split(train_val_df)]
    return splits, test_df

In [ ]:
# Sequence creation for multivariate time series
def create_sequences(df, lookback=LOOKBACK, horizon=HORIZON, static_X=static_X, temporal_X=temporal_X):
    X, y, anomaly, cell_id = [], [], [], []

    # Loop through each unique cell in the dataset
    for cell in df['cell'].unique():
        # Filter the dataframe for the current cell only
        cell_df = df[df['cell'] == cell]

        # Generate sequences within this cell's data
        for i in range(lookback, len(cell_df) - horizon + 1):
            # Lookback sequences with time-variant features
            X_seq = cell_df.iloc[i - lookback:i][['minRSSI'] + temporal_X].values

            # Repeat static features across lookback window and concatenate to time-variant features
            static_seq = cell_df.iloc[i][static_X].values  # Static features for this cell at a single timestep
            static_seq = np.tile(static_seq, (lookback, 1))  # Repeat to match lookback window length

            # Concatenate time-variant and time-invariant features
            X_combined = np.concatenate([X_seq, static_seq], axis=1)

            # Target horizon sequence
            y_seq = cell_df.iloc[i:i + horizon]['minRSSI'].values
            # Anomaly sequences for later evaluation
            anomaly_seq = cell_df.iloc[i:i + horizon]['anomaly'].values
            # Cell ID for each sequence
            cell_seq = cell_df.iloc[i:i + horizon]['cell'].values

            # Append sequences to output lists
            X.append(X_combined)
            y.append(y_seq)
            anomaly.append(anomaly_seq)
            cell_id.append(cell_seq)

    # Convert lists to numpy arrays for model input
    return np.array(X), np.array(y), np.array(anomaly), np.array(cell_id)

In [ ]:
def scale_data_split(train_df, val_df, temporal_features=temporal_X, static_features=static_X):
    scaler_temporal = StandardScaler()
    scaler_static = MinMaxScaler()
    scaler_target = StandardScaler()

    # Scale time-variant features
    train_df[temporal_features] = scaler_temporal.fit_transform(train_df[temporal_features])
    val_df[temporal_features] = scaler_temporal.transform(val_df[temporal_features])

    # Scale time-invariant features
    train_df[static_features] = scaler_static.fit_transform(train_df[static_features])
    val_df[static_features] = scaler_static.transform(val_df[static_features])

    # Scale minRSSI separately (target variable)
    train_df['minRSSI'] = scaler_target.fit_transform(train_df[['minRSSI']])
    val_df['minRSSI'] = scaler_target.transform(val_df[['minRSSI']])

    return train_df, val_df, scaler_target, scaler_temporal, scaler_static

In [ ]:
def tune_multi_lstm(X_train, y_train, max_epochs=EPOCHS):

    def build_tunable_lstm(hp):
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

        # Add LSTM layers with recurrent dropout
        for i in range(hp.Int('num_lstm_layers', min_value=1, max_value=4)):
            model.add(LSTM(
                units=hp.Choice('units', values=[10, 20, 50, 100]),
                activation=hp.Choice('activation', values=['relu', 'tanh']),
                return_sequences=True if i < hp.get('num_lstm_layers') - 1 else False,
                kernel_regularizer=l2(hp.Choice('l2_regularizer', values=[1e-2, 1e-3, 1e-4])),
                recurrent_dropout=hp.Float(f'recurrent_dropout_{i}', min_value=0.1, max_value=0.4, step=0.1)
            ))

        # Add dense layers
        for j in range(hp.Int('num_dense_layers', min_value=1, max_value=3)):
            model.add(Dense(units=hp.Choice(f'dense_units_{j}', values=[24, 48, 96, 288])))

            # Optional dense layer dropout
            if hp.Boolean(f'use_dense_dropout_{j}'):
                dense_dropout_rate = hp.Float(f'dense_dropout_rate_{j}', min_value=0.1, max_value=0.3, step=0.1)
                model.add(Dropout(dense_dropout_rate))

        model.add(Dense(HORIZON))

        # Compile model with fixed optimizer (Adam) and tunable loss function
        model.compile(
            optimizer='adam',
            loss=hp.Choice('loss', values=['mse', 'mae'])
        )

        # Define batch size as a tunable hyperparameter
        batch_size = hp.Choice('batch_size', [16, 32, 64, 128])

        return model

    # Hyperband tuner instance
    tuner = kt.Hyperband(
        hypermodel=build_tunable_lstm,
        objective='val_loss',
        max_epochs=max_epochs,
        factor=3,
        directory='/content/drive/MyDrive/Thesis/Thesis/lstm/multivar_tuning',
        project_name='lstm_tuning'
    )

    # Fit Hyperband tuner to training data
    tuner.search(X_train, y_train,
                 validation_data=(X_val, y_val),
                 epochs=max_epochs,
                 verbose=1)

    # Get best model and hyperparameters
    best_model = tuner.get_best_models(num_models=1)[0]
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0].values

    return best_model, best_params

# Tune

In [ ]:
splits, test_set = time_series_split(df, 4)

for i, (train, val) in enumerate(splits):
    print(f"Split {i + 1}:")
    print(f"  Train set shape: {train.shape}")
    print(f"  Validation set shape: {val.shape}")

print(f"Test set shape: {test_set.shape}")

Split 1:
  Train set shape: (149388, 24)
  Validation set shape: (149385, 24)
Split 2:
  Train set shape: (298773, 24)
  Validation set shape: (149385, 24)
Split 3:
  Train set shape: (448158, 24)
  Validation set shape: (149385, 24)
Split 4:
  Train set shape: (597543, 24)
  Validation set shape: (149385, 24)
Test set shape: (186733, 24)


In [ ]:
scalers = {}

split1, split2, split3, split4 = splits
scaled_train, scaled_val, scaler_target, scaler_temporal, scaler_static = scale_data_split(split3[0], split3[1])
scalers = {
    'scaler_target': scaler_target,
    'scaler_temporal': scaler_temporal,
    'scaler_static': scaler_static
}

# Create sequences
X_train, y_train, _, _ = create_sequences(scaled_train, LOOKBACK, HORIZON)
X_val, y_val, _, _ = create_sequences(scaled_val, LOOKBACK, HORIZON)

# Convert to float32 to avoid data type issues
X_train, y_train = X_train.astype(np.float32), y_train.astype(np.float32)
X_val, y_val = X_val.astype(np.float32), y_val.astype(np.float32)

In [ ]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (435139, 24, 13), y_train shape: (435139, 24)
X_val shape: (136366, 24, 13), y_val shape: (136366, 24)


In [ ]:
# Tune and train the model
best_model, best_params = tune_multi_lstm(X_train, y_train)
print(best_params)

Trial 30 Complete [00h 17m 36s]
val_loss: 0.3031729459762573

Best val_loss So Far: 0.2620237469673157
Total elapsed time: 1d 03h 51m 56s
{'num_lstm_layers': 1, 'units': 10, 'activation': 'relu', 'l2_regularizer': 0.001, 'recurrent_dropout_0': 0.30000000000000004, 'num_dense_layers': 1, 'dense_units_0': 96, 'use_dense_dropout_0': False, 'loss': 'mae', 'batch_size': 16, 'recurrent_dropout_1': 0.4, 'recurrent_dropout_2': 0.30000000000000004, 'recurrent_dropout_3': 0.30000000000000004, 'dense_units_1': 24, 'use_dense_dropout_1': True, 'dense_units_2': 288, 'use_dense_dropout_2': True, 'dense_dropout_rate_1': 0.1, 'dense_dropout_rate_0': 0.1, 'dense_dropout_rate_2': 0.2, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0014'}
